# ROBOFISH

### Imports

In [3]:
import FISH2_functions

Module ThermoCube not found. Ignore if machine is not in use
Module YoctoThermisto_FISH not found. Ignore if machine is not in use


### Find machine address
Use the below function to find the USB address and serial number of the connected devices, one device at a time.  
If you know the identifier of the FTDI chip of your device use it as input.  
Otherwise follow the instructions of the function and unplug and plug your device to find the address.  
  
When the search is successful add the serial number (preferred) or the USB COM port (alternative) to the data file.  
Open the data file with the `ROBOFISH_user_program.py` and add the info to the `ROBOFISH_System_datafile.yaml`.

In [4]:
FISH2_functions.find_address()

Performing manual USB address search.


    Unplug the USB. Press Enter if unplugged... 
    Plug in the USB. Press Enter if USB has been plugged in... 


    No port found. Try again.



    Do you want to try again? Y/N n


# Initiate system

In [ ]:
import FISH2_functions

#Path to the  database that is used to keep track of the experiment data. 
#database is automatically generated by the "FISH2_user_program.py" and prints the path after runing, copy the path to this location 
db_path = 'FISH_database\FISH_System2_db.sqlite'

#System specific path to start_imaging_file. If not present make a file called start_imaging_file.txt with a single 0.
#Then paste the path here, and also use this path for the imaging program.
start_imaging_file_path = "C:\\Users\\BL\\Desktop\\ROBOFISH\\FISH_database\\start_imaging_file.txt"

#System specific path to where the microscope saves the images. The info files will be put here.
#Use double slashes "\\" but do not put the trailing slashes! Example "C:\\Folder\\subfolder"
imaging_output_folder = "G:\\To_Monod\\LBEXP20210428_EEL_HE_3370um"


F2 = FISH2_functions.FISH2(db_path, imaging_output_folder, start_imaging_file_path, system_name='ROBOFISH')

# Basic functions

In [ ]:
# The below commands are for basic operation of the ROBOFISH system.
# Some functions are capable of more complex settings. Please see the documentation of each function for a description.
# This information is accessible in Jupyter Lab by hitting Shift+Tab while having the cursor between the brackets of the function.
# Or in the source file "FISH2_functions.py"

# Volumes are in microliter (ul)
# Temperatures are in degree Celcius (C)

# Write message to log file
F2.L.logger.info('Your message here')

# Reset the reservoir. Will refresh the requested amount of running buffer
F2.resetReservoir(100, update_buffer=True) 
    #Refreshing 100ul and updating the volume.

# Dispense some running buffer to the flow cell
F2.extractDispenseRunningBuffer(1000, 'Chamber1') 
    #Will dispence 1000ul to Chamber1

# Dispese a specfic buffer to the flow cell
F2.extractDispenseBuffer('buffer_name', 1000, 'Chamber1', padding=True)
    # "buffer_name" can be the name of the buffer or the port number.
    # Volume is 1000ul
    # Dispence to "Chamber1"
    # If padding is True the bufffer will be dispenced exactly to the flow cell.
    # The dead volume in the tubes between valve and flow cell will be bridged with running buffer.
    
# Dispense a hybridization mix with probes to the flow cell
F2.extractDispenseHybmix('Chamber1', 3, slow_speed=32, steps=0)
    # Dispence hybridization mix beloing to Chamber1 cycle 3 to Chamber1.
    # Because the hybmix is viscous the pumping speed should be slower, here speed setting 32 is used.
    # Check the respective code of the pump for the correct speed code.
    # The steps option can have the hybridization mix be pumped intermittendly through the degasser.
    # Use this if you get issues with air bubles. Alternatively you can lower the speed even more.

    
# Set the temperature of the flow cell
F2.setTemp(37, 'Chamber1')
    # Set temperature of Chamber1 to 37C.
    # If you want to acces the temperatures that you entered in the info file use the Parameters dictionary:
    # F2.Parameters['Imaging_temperature']
    
# Wait untill a specific temperature has been reached.
F2.waitTemp(37, 'Chamber1', error=3, sd=0.01, verbose=False)
    # Blocks untill 37C has been reached in Chamber1.
    # The measured temperature may deviate 3C from the target temperature.
    # The standard deviation (sd) should be below 0.01 for the temperature to be considered stable.
    # If verbose is True is will print the measured temperature each second.
    
# The extractDispence* functions can also take arguments that control the incubation time.
# They take the keyword agruments h for hours, m for minutes and s for seconds.
# Example:
F2.extractDispenseBuffer('buffer_name', 1000, 'Chamber1', padding=True, h=1, m=30)
    # This means that after the dispensing of the buffer the program will wait 1 hour and 30 minutes.
    # In other words the sample is incubated 1 hour 30 minutes with the dispensed buffer.
# The time it takes to execute the function is substracted from the incubation time if it does not take more than 10% of the incubation time.


# Use the above funtions to build a protocol.
# For osmFISH the protocol consists of a first round and then by a number of repeated rounds.
# You can build this as two functions.
#Example:
def first_round(chamber, cycle_number):
    #Your protocol for first round here
    print('Completed first round')
    
def repeat_round(chamber, cycle_number):
    #Your protocol for first round here
    print(f'Completed cycle {cycle_number}')
    
# If you have your two functions that take a chamber and cycle number argument,
# you can use the scheduler function to execute the full experiment for you.
F2.scheduler(first_round, repeat_round)
    # This function will first perform the first_round() function
    # Then it will send a message to the microscope to start the imaging. (This needs to be set up, see ROBOFISH github page)
    # When the imaging is done it will perform the repeat_round() function.
    # The total number of rounds and other settings are controlled by the info_file.